# **Modelo Temporal Previsão de Risco de Diabetes**

## **Carregamento e preparação dos dados**

In [16]:
import pandas as pd

#Lê o arquivo CSV
df = pd.read_csv(r'https://raw.githubusercontent.com/vitoriall/diabetes-prediction-raitec/refs/heads/main/data/dados_pacientes_diabetes%20temporal.csv')

#Converte a coluna 'data' para tipo datetime
df['data'] = pd.to_datetime(df['data'])

# Ordenar os dados por paciente e data
df = df.sort_values(by=['paciente_id', 'data'])

            data  paciente_id  idade sexo   imc  glicemia atividade_fisica  \
0     2023-01-01            1     77    M  31.5      90.6         moderada   
100   2023-01-02            1     77    M  31.5     117.9            baixa   
200   2023-01-03            1     77    M  31.5     106.3            baixa   
300   2023-01-04            1     77    M  31.5      97.1         moderada   
400   2023-01-05            1     77    M  31.5     108.7         moderada   
...          ...          ...    ...  ...   ...       ...              ...   
36099 2023-12-27          100     53    F  24.4     101.6            baixa   
36199 2023-12-28          100     53    F  24.4     114.8            baixa   
36299 2023-12-29          100     53    F  24.4      89.7            baixa   
36399 2023-12-30          100     53    F  24.4      94.5         moderada   
36499 2023-12-31          100     53    F  24.4      81.2            baixa   

      consumo_acucar  tem_diabetes  
0           moderado      

## **Criação de variáveis temporais (lags, médias móveis, diferenças)**

In [6]:
def add_temporal_features(group):
    #Copia o grupo para evitar avisos do pandas sobre alteração de dados em slices da tabela original.
    group = group.copy()

    #shift(n) pega o valor de glicemia de n dias atrás.
    #Isso permite capturar como a glicemia estava antes, e identificar aumentos ou quedas.
    group['glicemia_lag1'] = group['glicemia'].shift(1)
    group['glicemia_lag3'] = group['glicemia'].shift(3)
    group['glicemia_lag7'] = group['glicemia'].shift(7)

    #Isso calcula a média da glicemia nos últimos 3, 5 ou 7 registros.
    #Serve para suavizar a série e detectar tendências de forma mais robusta.
    group['glicemia_ma3'] = group['glicemia'].rolling(window=3).mean()
    group['glicemia_ma5'] = group['glicemia'].rolling(window=5).mean()
    group['glicemia_ma7'] = group['glicemia'].rolling(window=7).mean()

    # Captura a variação da glicemia entre hoje e 1 ou 3 dias atrás.
    group['glicemia_diff1'] = group['glicemia'] - group['glicemia_lag1']
    group['glicemia_diff3'] = group['glicemia'] - group['glicemia_lag3']

    return group
#Agrupa os dados por paciente_id, e aplica a função a cada paciente separadamente.
df = df.groupby('paciente_id').apply(add_temporal_features)

#Remoção de valores faltantes
df = df.dropna(subset=['glicemia_lag1', 'glicemia_ma3', 'glicemia_ma7', 'glicemia_diff1'])

<ipython-input-6-1814952757>:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('paciente_id').apply(add_temporal_features)


## **Criação do rótulo** alerta_risco

In [7]:
#A condição do alerta é ativada se:
#A glicemia atual for mais de 10% maior que a média móvel de 3 dias, OU
#A diferença da glicemia de hoje em relação a ontem (diff1) for maior que 20
df['alerta_risco'] = ((df['glicemia'] > df['glicemia_ma3'] * 1.1) | (df['glicemia_diff1'] > 20)).astype(int)

#Calcula a proporção de linhas com alerta (valor médio da coluna que só tem 0 e 1)
alerta_rate = df['alerta_risco'].mean()


df[['data', 'paciente_id', 'glicemia', 'glicemia_lag1', 'alerta_risco']].head(), alerta_rate

(                       data  paciente_id  glicemia  glicemia_lag1  \
 paciente_id                                                         
 1           600  2023-01-07            1      82.2          101.8   
             700  2023-01-08            1      92.1           82.2   
             800  2023-01-09            1      98.3           92.1   
             900  2023-01-10            1     132.4           98.3   
             1000 2023-01-11            1     104.3          132.4   
 
                   alerta_risco  
 paciente_id                     
 1           600              0  
             700              0  
             800              0  
             900              1  
             1000             0  ,
 np.float64(0.2050974930362117))

## **Criação de variáveis preditoras e normalização**

In [8]:
from sklearn.preprocessing import StandardScaler

features = ['idade', 'imc', 'glicemia', 'glicemia_lag1', 'glicemia_ma3', 'glicemia_ma7']
X = df[features]
y = df['alerta_risco']
X_indices = X.index  # <- guarda os índices originais

# Padronização dos dados
#Isso não muda a relação entre os dados, mas garante que todas as variáveis estejam na mesma escala, o que é essencial para muitos algoritmos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

## **Divisão em treino e teste + treinamento**

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

#stratify=y: garante que a proporção de alertas (0s e 1s em y) seja a mesma em treino e teste.
X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(
    X_scaled, y, X_indices, test_size=0.2, stratify=y, random_state=42
)
#Também divide os índices (X_indices) para que você possa depois identificar quais linhas do dataframe original estão em cada conjunto.


model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

## **Avaliação do modelo**

In [15]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = model.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[5679   28]
 [  55 1418]]
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5707
           1       0.98      0.96      0.97      1473

    accuracy                           0.99      7180
   macro avg       0.99      0.98      0.98      7180
weighted avg       0.99      0.99      0.99      7180



## **Gerar alertas previstos com base nas previsões do modelo**

In [14]:
#uso dos índices originais (idx_test) para resgatar as linhas correspondentes no DataFrame original (df).
df_test = df.loc[idx_test]
df_test['alerta_previsto'] = model.predict(X_test)

# Exibir alertas por paciente
alertas = df_test[df_test['alerta_previsto'] == 1][['data', 'paciente_id', 'glicemia']]
print(alertas.head())

                        data  paciente_id  glicemia
paciente_id                                        
72          21471 2023-08-03           72     113.2
21          9720  2023-04-08           21     121.2
12          6611  2023-03-08           12      98.2
76          34575 2023-12-12           76      92.8
69          14668 2023-05-27           69      92.8
...                      ...          ...       ...
38          31837 2023-11-15           38     110.7
14          2813  2023-01-29           14     100.4
16          27915 2023-10-07           16     119.3
18          26717 2023-09-25           18     111.1
80          18179 2023-07-01           80     116.2

[200 rows x 3 columns]


**Possíveis melhorias:**
Usar modelos temporais mais específicos(LSTM);
Ajustar thresholds da regra de alerta para mais sensibilidade ou especificidade